<center><h1> Tarea 1 Machine Learning </h1></center>
<center><h3> Matías Ramírez Marianetti | Javier Rodríguez Aguilera</h3></center>
<center><h3> 201173506-4 | 201173553-6 </h3></center>
<center><h3> matias.ramirezm@alumnos.usm.cl | javier.rodriguez@alumnos.usm.cl </h3></center>

<h3>Pregunta 1</h3>

Se utilizará un pequeño dataset de nombre prostate-cancer el cual es usado con frecuencia para realizar pruebas priliminares de con métodos de regresión. Estos datos consisten en la eventual correlación entre el nivel de antígeno prostático específico medido de un paciente, y otra serie de otras mediciones clínicas obtenidas después de practicar al paciente una prostatectonía radical.

<h4> a) </h4>

In [6]:
import pandas as pd
import numpy as np
url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
df = df.drop('Unnamed: 0', axis=1)
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)
df = df.drop('train', axis=1)

#print df

Para comenzar entre las lineas cinco(5) y nueve(9) de este algoritmo se elimina la columna que contiene el número de cada registro del dataset, de esta forma no se tendrá una variable innecesaria. Luego se toma la columna 'train' y a través de esta se generan dos arreglos, istrain e istest. En la línea de istrain se recorre la columna 'train' contenida en la variable istrain_str y se le coloca True a los datos en que 'train' sea 'T' y False en el caso contrario, de esta forma para el caso de istest se toma istrain y se invierten los valores booleanos de este. Finalmente se elimina la columna 'train', ya que sus valores serán utilizados a través de los arreglos anteriormente señalados.

<h4> b) </h4>

In [24]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 6.9 KB


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


<h4> c) </h4>

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']

# print df_scaled

La utilidad de normalizar los datos es arreglar las varianzas y de esta forma dejar todas respecto a un mismo punto.

<h4>d)</h4>

In [9]:
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

Lo importante del tercer paso en este caso, es agregar una nueva columna que contiene solamente unos(1), los cuales denotan a $x^{(0)}$ y de esta forma definir un modelo lineal en forma matricial.

<h4> e) </h4>

In [10]:
#print linreg.coef_

cols = list(Xtrain.columns)
cols.remove("intercept")

Xtrain_zscores = pd.DataFrame()

for col in cols:
    col_zscore = col + '_zscore'
    Xtrain_zscores[col_zscore] = (Xtrain[col] - Xtrain[col].mean())/Xtrain[col].std(ddof=0)
    
#print Xtrain_zscores

In [11]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
from sklearn import cross_validation
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
k_fold = cross_validation.KFold(len(Xm),10)
mse_cv = 0
for k, (train, val) in enumerate(k_fold):
    linreg = lm.LinearRegression(fit_intercept = False)
    linreg.fit(Xm[train], ym[train])
    yhat_val = linreg.predict(Xm[val])
    mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
    mse_cv += mse_fold
mse_cv = mse_cv / 10

Para lograr definir la variable más importante, se considera la que tiene el mayor peso $\beta$. En el caso del valor del z-score, este estandarizará los valores y de esta forma los dejará a todos en un rango cercano, el $95\%$ debería estar entre $-2$ y $2$ (desviaciones estándares del promedio) y así los valores del peso tendrán relevancia sin importar si en el dataset hay valores que tienen valores muy altos.

<h4> f) </h>

In [ ]:
#Agregar código folds

En este caso se obtuvieron dos errores numéricos, los cuales luego de haber calculado su error porcentual, $84,36\%$ y $72,32\%$ para $5$ y $10$ folds respectivamente, se puede decir que son bastante altos. A medida que se van aumentando los folds este error irá disminuyendo debido a que la cantidad valores de entrenamiento aumentará por lo que el modelo se ajustará de mejor manera. 
El error del modelo sobre el conjunto de prueba es $0,5$, con un error porcentual de $22\%$. 

<h4> j) </h4>

# 2

## a)

In [12]:
def fss(x, y, names_x, k = 10000):
    p = x.shape[1]-1
    k = min(p, k)
    names_x = np.array(names_x)
    remaining = range(0, p)
    selected = [p]
    current_score = 0.0
    best_new_score = 0.0
    while remaining and len(selected)<=k :
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            x_train = x[:,indexes]
            predictions_train = model.fit(x_train, y).predict(x_train)
            residuals_train = predictions_train - y
            mse_candidate = np.mean(np.power(residuals_train, 2))
            score_candidates.append((mse_candidate, candidate))
        
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        best_new_score, best_candidate = score_candidates.pop()
        remaining.remove(best_candidate)

In [16]:
fss(X,ytrain,)

TypeError: unhashable type

## b)